In [1]:
import pandas as pd
from tqdm.auto import tqdm
import json
from pathlib import Path

p_narr = Path('/data/gunsbrother/prjs/ltvu/tasks/Ego4D/annotations/narration.json')
p_narr_red = Path('/data/gunsbrother/prjs/ltvu/tasks/Ego4D/annotations/all_narrations_redacted.json')

In [2]:
data = json.load(p_narr_red.open())

In [3]:
l = len(data['videos'])
l

8922

In [14]:
f, fr = p_narr.open('rb'), p_narr_red.open('rb')
p_df_dir = Path('/data/gunsbrother/prjs/ltvu/ours/data/ego4d')
p_df_dir.mkdir(exist_ok=True, parents=True)
p_csv = p_df_dir / 'narrations.csv'
f_csv = p_csv.open('w')
p_pkl_dir = p_df_dir / 'narrations/pkls'
p_pkl_dir.mkdir(exist_ok=True, parents=True)
for idx, (video_id, narr_obj) in tqdm(enumerate(data['videos'].items()), total=8922):
    narrs = narr_obj['narrations']
    if narrs:
        df = pd.DataFrame.from_records(narrs, columns=['_annotation_uid', 'annotator', 'time', 'text', '_project_id'])
        df['time'] = df['time'].astype('f4')
        df['text'] = df['text'].str.replace('\n', ' ').str.strip().str.replace('  ', ' ')
        df.insert(0, 'video_uid', video_id)
        df.to_csv(f_csv, header=idx==0, index=False, float_format='%.5f')
        with (p_pkl_dir / f'{video_id}.pkl').open('wb') as f_pkl:
            df.to_pickle(f_pkl)
f_csv.close(), f_pkl.close()
df.head()

  0%|          | 0/8922 [00:00<?, ?it/s]

,video_uid,_annotation_uid,annotator,time,text,_project_id
0,c4892805-56e3-4509-8a30-55c4fc38e4ac,dbdb3fb7-96db-4f74-ab6a-9cb62ab335b7,1980142755361387,0.583788,#C C picks a bunch of clips.,3111279962304712
1,c4892805-56e3-4509-8a30-55c4fc38e4ac,a9e261b9-52a9-4b78-a498-2f02144e3b5c,4577349869004223,0.862128,#C C picks up a bunch of cable ties from a table,347545199711956
2,c4892805-56e3-4509-8a30-55c4fc38e4ac,dbdb3fb7-96db-4f74-ab6a-9cb62ab335b7,1980142755361387,1.511248,#C C holds the clips with his other hand.,3111279962304712
3,c4892805-56e3-4509-8a30-55c4fc38e4ac,a9e261b9-52a9-4b78-a498-2f02144e3b5c,4577349869004223,1.871458,#C C walks towards a couple of cables on the wall,347545199711956
4,c4892805-56e3-4509-8a30-55c4fc38e4ac,dbdb3fb7-96db-4f74-ab6a-9cb62ab335b7,1980142755361387,1.967548,#C C moves around the room.,3111279962304712


# 만들고 나서 통계 확인

In [3]:
import pandas as pd
from pathlib import Path

p_csv = Path('/data/gunsbrother/prjs/ltvu/ours/data/ego4d/narrations.csv')
df = pd.read_csv(p_csv)
df.head()

,video_uid,_annotation_uid,annotator,time,text,_project_id
0,cba13243-5ad3-4360-87df-6286867c473a,8a908b99-0e31-4427-aa93-c4a2509bbb88,1695609517163352,0.04206,#C C converses with man X,3111279962304712
1,cba13243-5ad3-4360-87df-6286867c473a,8a908b99-0e31-4427-aa93-c4a2509bbb88,1695609517163352,0.79255,#C C looks at the green vegetables,3111279962304712
2,cba13243-5ad3-4360-87df-6286867c473a,d47d1694-9461-4e8f-9601-2c8ccc3884ee,4288081301262926,11.89834,#C C cuts vegetables on a chair with her hands.,347545199711956
3,cba13243-5ad3-4360-87df-6286867c473a,d47d1694-9461-4e8f-9601-2c8ccc3884ee,4288081301262926,13.07206,#C C drops the vegetable on the chair with oth...,347545199711956
4,cba13243-5ad3-4360-87df-6286867c473a,8a908b99-0e31-4427-aa93-c4a2509bbb88,1695609517163352,13.83165,#C C separates green vegetables from their stalks,3111279962304712


In [9]:
g = df.groupby('video_uid')
g.agg({'time': ['min', 'mean', 'max']})

time                         
                                          min         mean         max
video_uid                                                             
000786a7-3f9d-4fe6-bfb3-045b368f7d44  0.00000   176.249861   414.58829
000cd456-ff8d-499b-b0c1-4acead128a8b  0.02806   383.273864   956.74664
001e3e4e-2743-47fc-8564-d5efd11f9e90  0.02806    53.829214   121.74184
00277df3-9107-4592-ba85-b8d054149551  0.22088    34.201785    86.75726
002ad105-bd9a-4858-953e-54e88dc7587e  0.00872   398.739377   918.76904
...                                       ...          ...         ...
ffdb7654-1b86-4cc6-9453-834bbe3361aa  0.68783   332.810403   589.90051
ffe22125-67ae-4ee1-9f11-2b2df706d1b6  0.00000   779.250285  1688.97375
ffefe048-8408-4444-83e6-f05a4adcdcf0  0.00000    51.237320   140.76717
fff5b8bd-1fc2-457b-9760-7691a7c1d095  0.00000  1013.651292  1830.00269
fffbaeef-577f-45f0-baa9-f10cabf62dfb  0.03203   382.216934   738.57495

[8873 rows x 3 columns]

In [10]:
g.agg('size')

video_uid
000786a7-3f9d-4fe6-bfb3-045b368f7d44     186
000cd456-ff8d-499b-b0c1-4acead128a8b     282
001e3e4e-2743-47fc-8564-d5efd11f9e90     108
00277df3-9107-4592-ba85-b8d054149551      43
002ad105-bd9a-4858-953e-54e88dc7587e     244
                                        ... 
ffdb7654-1b86-4cc6-9453-834bbe3361aa     548
ffe22125-67ae-4ee1-9f11-2b2df706d1b6    1195
ffefe048-8408-4444-83e6-f05a4adcdcf0     128
fff5b8bd-1fc2-457b-9760-7691a7c1d095     317
fffbaeef-577f-45f0-baa9-f10cabf62dfb     240
Length: 8873, dtype: int64

# Video info

In [13]:
data_videoinfo = json.load(open('/data/datasets/ego4d_data/ego4d.json'))
data_videoinfo.keys()

dict_keys(['date', 'version', 'description', 'videos', 'clips', 'concurrent_video_sets', 'physical_settings', 'moments_labels'])

In [14]:
target_keys_video = [
    'video_uid',
    'duration_sec',
    'scenarios',  # list[str]
    'video_metadata.fps',
    'video_metadata.num_frames',
    'split_em',  # nullable
    'split_av',  # nullable
    'split_fho',  # nullable
    'has_imu',
    'has_gaze',
]

target_keys_clip = [
    'clip_uid',
    'video_uid',
    'video_start_sec',
    'video_end_sec',
    'video_start_frame',
    'video_end_frame',
    'clip_metadata.mp4_duration_sec',
    'clip_metadata.num_frames',
]

In [12]:
data_videoinfo['videos'][0]

{'video_uid': '77cc4654-4eec-44c6-af05-dbdf71f9a401',
 'duration_sec': 1247.13,
 'scenarios': ['Car - commuting, road trip', 'Talking with family members'],
 'video_metadata': {'fps': 30.0,
  'num_frames': 37414,
  'video_codec': 'libvpx-vp9',
  'audio_codec': 'aac',
  'display_resolution_width': 1920,
  'display_resolution_height': 1080,
  'sample_resolution_width': 1920,
  'sample_resolution_height': 1080,
  'mp4_duration_sec': 1247.13,
  'video_start_sec': 0.0,
  'video_duration_sec': 1247.13,
  'audio_start_sec': None,
  'audio_duration_sec': None,
  'video_start_pts': 0,
  'video_duration_pts': 19155968,
  'video_base_numerator': 1,
  'video_base_denominator': 15360,
  'audio_start_pts': None,
  'audio_duration_pts': None,
  'audio_base_numerator': None,
  'audio_base_denominator': None},
 'split_em': None,
 'split_av': None,
 'split_fho': None,
 's3_path': 's3://ego4d-minnesota/public/v1/full_scale/77cc4654-4eec-44c6-af05-dbdf71f9a401',
 'manifold_path': None,
 'origin_video_id':